In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import json

import os
import openai
from tqdm.notebook import tqdm

from image_utils import crop_bbox

In [3]:
df = pd.read_csv("../data/imaterialist-fashion-2020-fgvc7/clothes_final.csv")

In [4]:
df.head(3)

,ImageId,EncodedPixels,Height,Width,ClassId,AttributesIds,second_AttributesIds,bbox,bbox_big,width,height,area,id,name,supercategory,AttributesNames,second_AttributesNames
0,00000663ed1ff0c4e0132b9b9ac53f6e,6068157 7 6073371 20 6078584 34 6083797 48 608...,5214,3676,6,"115,136,143,154,230,295,316,317",NaN,"[1163, 3923, 2839, 5212]","(1122, 3891, 2880, 5214)",1758,1323,2325834,6,pants,lowerbody,"symmetrical, regular (fit), low waist, maxi (l...",NaN
1,00000663ed1ff0c4e0132b9b9ac53f6e,6323163 11 6328356 32 6333549 53 6338742 75 63...,5214,3676,0,"115,136,142,146,225,295,316,317",163,"[1212, 1371, 2394, 3978]","(1183, 1306, 2423, 4043)",1240,2737,3393880,0,"shirt, blouse",upperbody,"symmetrical, regular (fit), normal waist, abov...",shirt (collar)
2,00000663ed1ff0c4e0132b9b9ac53f6e,4566382 8 4571592 25 4576803 41 4582013 58 458...,5214,3676,4,"17,115,136,145,149,225,295,311,317","219,204,160,174","[875, 1437, 3309, 4871]","(815, 1352, 3369, 4956)",2554,3604,9204616,4,jacket,upperbody,"blazer, symmetrical, regular (fit), no waistli...","welt (pocket), set-in sleeve, wrist-length, no..."


In [5]:
df['bbox'][0]

'[1163, 3923, 2839, 5212]'

In [6]:
def listify(string, encap_type="()"):
    '''
    string 값들을 list 화
    '''
    return [int(num) for num in string.strip(encap_type).split(', ')]

In [8]:
# 처음 읽을 때, pandas dataframe에서 list가 아닌 string 값으로 인식하기 때문에 변환 필요
df['bbox'] = [listify(i, encap_type="[]") for i in df['bbox']]
df['bbox_big'] = [listify(i) for i in df['bbox_big']]

In [9]:
df.head(2)

,ImageId,EncodedPixels,Height,Width,ClassId,AttributesIds,second_AttributesIds,bbox,bbox_big,width,height,area,id,name,supercategory,AttributesNames,second_AttributesNames
0,00000663ed1ff0c4e0132b9b9ac53f6e,6068157 7 6073371 20 6078584 34 6083797 48 608...,5214,3676,6,"115,136,143,154,230,295,316,317",NaN,"[1163, 3923, 2839, 5212]","[1122, 3891, 2880, 5214]",1758,1323,2325834,6,pants,lowerbody,"symmetrical, regular (fit), low waist, maxi (l...",NaN
1,00000663ed1ff0c4e0132b9b9ac53f6e,6323163 11 6328356 32 6333549 53 6338742 75 63...,5214,3676,0,"115,136,142,146,225,295,316,317",163,"[1212, 1371, 2394, 3978]","[1183, 1306, 2423, 4043]",1240,2737,3393880,0,"shirt, blouse",upperbody,"symmetrical, regular (fit), normal waist, abov...",shirt (collar)


# CLIP embeddings

- Local에 각 상품별 cropped image 저장
- CLIP embeddings 생성

---

## 1. Bounding box EDA
- 각 이미지에 들어있는 상품들의 '크기'는?
- 유사도는 이미지의 크기에도 영향을 받기 때문에 중요한 요인 중 하나
- 따라서 한 카테고리 내에 속하는 이미지들은 모두 동일한 크기로 표현되는 것이 중요

In [10]:
img = Image.open("../data/imaterialist-fashion-2020-fgvc7/train/007e66e7c2864eb3c1ef95cd3ab52687.jpg")

In [12]:
# img

In [13]:
df.loc[218]

ImageId                                    007e66e7c2864eb3c1ef95cd3ab52687
EncodedPixels             5093384 3 5098565 11 5103747 17 5108928 25 511...
Height                                                                 5184
Width                                                                  3456
ClassId                                                                   2
AttributesIds                               115,136,145,146,295,316,317,328
second_AttributesIds                                                204,160
bbox                                                [982, 1247, 2484, 3049]
bbox_big                                            [945, 1202, 2521, 3094]
width                                                                  1576
height                                                                 1892
area                                                                2981792
id                                                                        2
name        

In [15]:
cropped = crop_bbox(img, df['bbox'][218])
# cropped

In [ ]:
df.loc[223]

In [16]:
img = Image.open("../data/imaterialist-fashion-2020-fgvc7/train/007e66e7c2864eb3c1ef95cd3ab52687.jpg")
cropped = crop_bbox(img, df['bbox'][223])
# cropped

In [17]:
# 카테고리별 crop할 기준 확인

for cat in df['supercategory'].unique():
    tmp = df.loc[df['supercategory']==cat]
    print(cat)
    print(tmp['name'].unique())
    print("Area : {}, width : {}, height : {}".format(np.median(tmp['area']), np.median(tmp['width']), np.median(tmp['height'])))
    print("-"*10)

lowerbody
['pants' 'skirt' 'shorts']
Area : 215033.0, width : 416.0, height : 539.0
----------
upperbody
['shirt, blouse' 'jacket' 'top, t-shirt, sweatshirt' 'sweater' 'vest'
 'cardigan']
Area : 338575.0, width : 498.0, height : 691.0
----------
wholebody
['dress' 'coat' 'cape' 'jumpsuit']
Area : 406373.0, width : 478.0, height : 877.0
----------
legs and feet
['shoe' 'sock' 'tights, stockings' 'leg warmer']
Area : 13851.0, width : 100.0, height : 143.0
----------
head
['glasses' 'headband, head covering, hair accessory' 'hat']
Area : 14508.0, width : 156.0, height : 95.0
----------
others
['bag, wallet' 'scarf' 'umbrella']
Area : 64343.0, width : 196.0, height : 345.0
----------
waist
['belt']
Area : 21033.0, width : 195.0, height : 102.0
----------
arms and hands
['watch' 'glove']
Area : 5454.0, width : 72.0, height : 75.0
----------
neck
['tie']
Area : 20720.0, width : 123.0, height : 200.0
----------


각 카테고리 별로 이미지들의 특징을 갖고 있음
- lower body는 가로 평균 410, 세로 540
- upper body는 lower body보다 세로 비율이 더 길다
- wholebody는 그보다 세로 비율이 더 길다
- waist는 기로가 세로보다 더 길다
- arms and hands는 가로 세로 비율이 비슷하며, 전체적으로 작음

## 2. Bounding box를 기준으로 각 eneity를 crop

In [18]:
size = {"lowerbody":[420, 540],
        "upperbody":[500, 700],
        "wholebody":[480, 880],
        "legs and feet":[100, 150],
        "head":[150, 100],
        "others":[200, 350],
        "waist":[200, 100],
        "arms and hands":[75, 75],
        "neck":[120, 200]}

In [19]:
df.head(3)

,ImageId,EncodedPixels,Height,Width,ClassId,AttributesIds,second_AttributesIds,bbox,bbox_big,width,height,area,id,name,supercategory,AttributesNames,second_AttributesNames
0,00000663ed1ff0c4e0132b9b9ac53f6e,6068157 7 6073371 20 6078584 34 6083797 48 608...,5214,3676,6,"115,136,143,154,230,295,316,317",NaN,"[1163, 3923, 2839, 5212]","[1122, 3891, 2880, 5214]",1758,1323,2325834,6,pants,lowerbody,"symmetrical, regular (fit), low waist, maxi (l...",NaN
1,00000663ed1ff0c4e0132b9b9ac53f6e,6323163 11 6328356 32 6333549 53 6338742 75 63...,5214,3676,0,"115,136,142,146,225,295,316,317",163,"[1212, 1371, 2394, 3978]","[1183, 1306, 2423, 4043]",1240,2737,3393880,0,"shirt, blouse",upperbody,"symmetrical, regular (fit), normal waist, abov...",shirt (collar)
2,00000663ed1ff0c4e0132b9b9ac53f6e,4566382 8 4571592 25 4576803 41 4582013 58 458...,5214,3676,4,"17,115,136,145,149,225,295,311,317","219,204,160,174","[875, 1437, 3309, 4871]","[815, 1352, 3369, 4956]",2554,3604,9204616,4,jacket,upperbody,"blazer, symmetrical, regular (fit), no waistli...","welt (pocket), set-in sleeve, wrist-length, no..."


In [21]:
img = Image.open("../data/imaterialist-fashion-2020-fgvc7/train/00000663ed1ff0c4e0132b9b9ac53f6e.jpg")
cropped = crop_bbox(img, df['bbox_big'][0])
# cropped

#### 이미지 resize

In [22]:
from PIL import Image, ImageFilter

def resize_img(image, standard_size, category):
    w, h = image.size
    img_size = w*h

    new_width, new_height = standard_size[category]
    new_size = new_width * new_height

    if img_size >= new_size:
        # For downsizing
        downsized_image = image.resize((new_width, new_height), Image.Resampling.LANCZOS)
        return downsized_image
    else:
        # For upsizing
        upsized_image = image.resize((new_width, new_height), Image.Resampling.LANCZOS)
        upsized_image = upsized_image.filter(ImageFilter.UnsharpMask(radius=2, percent=150, threshold=3))
        return upsized_image

In [24]:
# resize_img(cropped, size, df['supercategory'][0])

## 3. Cropping 된 이미지를 각 항목에 따라 resize 후 로컬에 저장

약 40분 소요

In [26]:
base_path = "../data/imaterialist-fashion-2020-fgvc7/train"
cropped_path = "../data/imaterialist-fashion-2020-fgvc7/cropped_images"
new_df = pd.DataFrame()

for image_name in tqdm(df['ImageId'].unique()):
    # 한 이미지와 관련된 dataframe
    tmp = df.loc[df['ImageId']==image_name]
    tmp = tmp.reset_index().rename(columns={"index":"entity_id"})
    image = Image.open(os.path.join(base_path, image_name+".jpg"))
    # 각 이미지 내에 있는 상품들을 crop -> local save
    for idx, row in tmp.iterrows():
        cropped_img = crop_bbox(image, row['bbox_big'])
        resized_img = resize_img(cropped_img, size, row['supercategory'])
        resized_img.save(os.path.join(cropped_path, image_name + "_" + str(row['entity_id']) + ".jpg"))

    new_df = pd.concat([new_df, tmp], axis=0)

  0%|          | 0/27375 [00:00<?, ?it/s]

In [27]:
# new_df.to_csv("../data/imaterialist-fashion-2020-fgvc7/clothes_final2.csv", index=False)

In [28]:
new_df = pd.read_csv("../data/imaterialist-fashion-2020-fgvc7/clothes_final2.csv")

new_df['bbox'] = [listify(i, "[]") for i in new_df['bbox']]
new_df['bbox_big'] = [listify(i, "[]") for i in new_df['bbox_big']]

In [29]:
df.head(2)

,ImageId,EncodedPixels,Height,Width,ClassId,AttributesIds,second_AttributesIds,bbox,bbox_big,width,height,area,id,name,supercategory,AttributesNames,second_AttributesNames
0,00000663ed1ff0c4e0132b9b9ac53f6e,6068157 7 6073371 20 6078584 34 6083797 48 608...,5214,3676,6,"115,136,143,154,230,295,316,317",NaN,"[1163, 3923, 2839, 5212]","[1122, 3891, 2880, 5214]",1758,1323,2325834,6,pants,lowerbody,"symmetrical, regular (fit), low waist, maxi (l...",NaN
1,00000663ed1ff0c4e0132b9b9ac53f6e,6323163 11 6328356 32 6333549 53 6338742 75 63...,5214,3676,0,"115,136,142,146,225,295,316,317",163,"[1212, 1371, 2394, 3978]","[1183, 1306, 2423, 4043]",1240,2737,3393880,0,"shirt, blouse",upperbody,"symmetrical, regular (fit), normal waist, abov...",shirt (collar)


In [30]:
new_df.head()

,entity_id,ImageId,EncodedPixels,Height,Width,ClassId,AttributesIds,second_AttributesIds,bbox,bbox_big,width,height,area,id,name,supercategory,AttributesNames,second_AttributesNames
0,0,00000663ed1ff0c4e0132b9b9ac53f6e,6068157 7 6073371 20 6078584 34 6083797 48 608...,5214,3676,6,"115,136,143,154,230,295,316,317",NaN,"[1163, 3923, 2839, 5212]","[1122, 3891, 2880, 5214]",1758,1323,2325834,6,pants,lowerbody,"symmetrical, regular (fit), low waist, maxi (l...",NaN
1,1,00000663ed1ff0c4e0132b9b9ac53f6e,6323163 11 6328356 32 6333549 53 6338742 75 63...,5214,3676,0,"115,136,142,146,225,295,316,317",163,"[1212, 1371, 2394, 3978]","[1183, 1306, 2423, 4043]",1240,2737,3393880,0,"shirt, blouse",upperbody,"symmetrical, regular (fit), normal waist, abov...",shirt (collar)
2,2,00000663ed1ff0c4e0132b9b9ac53f6e,4566382 8 4571592 25 4576803 41 4582013 58 458...,5214,3676,4,"17,115,136,145,149,225,295,311,317","219,204,160,174","[875, 1437, 3309, 4871]","[815, 1352, 3369, 4956]",2554,3604,9204616,4,jacket,upperbody,"blazer, symmetrical, regular (fit), no waistli...","welt (pocket), set-in sleeve, wrist-length, no..."
3,3,0000fe7c9191fba733c8a69cfaf962b7,1343707 9 1346138 27 1348569 44 1351000 62 135...,2448,2448,1,"0,115,145,146,295,316,317",190,"[548, 405, 1946, 2263]","[514, 359, 1980, 2309]",1466,1950,2858700,1,"top, t-shirt, sweatshirt",upperbody,"classic (t-shirt), symmetrical, no waistline, ...",scoop (neck)
4,4,0002ec21ddb8477e98b2cbb87ea2e269,2287509 4 2290504 12 2290588 26 2293501 18 229...,3000,1997,10,"102,128,142,150,229,295,301,318",182,"[762, 719, 1262, 1967]","[750, 688, 1274, 1998]",524,1310,686440,10,dress,wholebody,"sheath (dress), straight, normal waist, above-...",round (neck)


## 4. CLIP을 활용한 embedding

- fashion 데이터셋을 활용하여 pretrain된 CLIP 모델
- CLIP 모델은 <이미지>-<caption> pair를 input data로 사용, 두 개를 하나의 동일한 embedding space에 구현
- 따라서 <패션 이미지>-<패션 caption> pair를 활용하여 fine-tuned된 모델이 현재 프로젝트 목적에 적합
- dot product를 사용하여 embedding ranking을 측정할 예정
```json
"FashionCLIP performs the dot product between the input caption embedding and each image vector embedding"

"The text used is a concatenation of the highlight (e.g., “stripes”, “long sleeves”, “Armani”) and short description (“80s styled t-shirt”)) available in the Farfetch dataset."
```

[Fine-tune 훈련 데이터](https://media.springernature.com/full/springer-static/image/art%3A10.1038%2Fs41598-022-23052-9/MediaObjects/41598_2022_23052_Fig3_HTML.png?as=webp, "Fine-tune 훈련 데이터")

(출처 : Contrastive language and vision learning of general fashion concepts)

- hugging face : https://huggingface.co/patrickjohncyh/fashion-clip
- paper : https://www.nature.com/articles/s41598-022-23052-9

#### F-CLIP VS CLIP 성능 차이

https://www.nature.com/articles/s41598-022-23052-9/tables/1

- HIT@5 = (서치 결과 top5에 있는 연관 상품의 개수) / (총 연관 상품의 개수)

In [32]:
# new_df.head()

In [33]:
from transformers import CLIPProcessor, CLIPModel

model_name = "patrickjohncyh/fashion-clip"
model = CLIPModel.from_pretrained(model_name)
processor = CLIPProcessor.from_pretrained(model_name)

config.json:   0%|          | 0.00/4.46k [00:00<?, ?B/s]

C:\Users\sckim\.conda\envs\vectordb\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\sckim\.cache\huggingface\hub\models--patrickjohncyh--fashion-clip. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/568 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [34]:
# crop된 이미지들의 path 불러오기
cropped_path = "../data/imaterialist-fashion-2020-fgvc7/cropped_images"

images = list(os.walk(cropped_path))[0][2]

In [35]:
images[:3]

['00000663ed1ff0c4e0132b9b9ac53f6e_0.jpg',
 '00000663ed1ff0c4e0132b9b9ac53f6e_1.jpg',
 '00000663ed1ff0c4e0132b9b9ac53f6e_2.jpg']

image embeddings from CLIP

In [36]:
from image_utils import extract_img_features

img_emb = extract_img_features(img, processor, model)

In [37]:
img_emb.shape

torch.Size([1, 512])

- `01.Create_image_embeddings.py` 참고

text embeddings from CLIP

In [ ]:
from transformers import CLIPProcessor, CLIPModel, AutoTokenizer

# model_name = "patrickjohncyh/fashion-clip"

model = CLIPModel.from_pretrained(model_name)
processor = CLIPProcessor.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
def get_single_text_embedding(text, model, tokenizer):
    inputs = tokenizer(text, return_tensors = "pt", padding=True)
    text_embeddings = model.get_text_features(**inputs)
    # convert the embeddings to numpy array
    embedding_as_np = text_embeddings.cpu().detach().numpy()
    return embedding_as_np.tolist()

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
img = Image.open("../data/imaterialist-fashion-2020-fgvc7/train/3bccf2e618d8f5f51442037ad3c8d4fb.jpg")
img

fashion fine-tuned model

```json
"The text used is a concatenation of the highlight (e.g., “stripes”, “long sleeves”, “Armani”) and short description (“80s styled t-shirt”)) available in the Farfetch dataset."
```

In [ ]:
img_emb = extract_img_features(img, processor, model)

sample_texts = ['tshirt', "formal suit and tie", 
                'a woman', "a lion in a cage", "black top short sleeves",
                'black shirt with check patterns, topwear', 'iphone']

sample_texts_emb = get_single_text_embedding(sample_texts, model, tokenizer)

sims = cosine_similarity(img_emb.cpu().detach().numpy(), sample_texts_emb)
# 이후세는 dot product를 사용 예정 
print("이미지와의 유사도")
for t, s in zip(sample_texts, sims[0]):
    print("{} : {}".format(t, s))
    print()

In [ ]:
img_emb.cpu().detach().numpy()[0].shape

In [ ]:
np.array(s).shape

In [ ]:
print('dot product')
for text, s in zip(sample_texts, sample_texts_emb):
    sim = np.dot(img_emb.cpu().detach().numpy()[0], np.array(s))
    print(text, sim)

---

In [ ]:
embeddings = {}

with open('../data/imaterialist-fashion-2020-fgvc7/img_embeddings_fashion_fine_tuned.json', 'r') as file:
    for line in file:
        # Convert each line to a dictionary
        embedding_dict = json.loads(line.strip())
        
        # Convert the list back to a NumPy array if necessary
        for img_name, emb_list in embedding_dict.items():
            embeddings[img_name] = np.array(emb_list)

In [ ]:
len(embeddings)

In [ ]:
type(embeddings)

In [ ]:
for k,v in embeddings.items():
    print(k)
    break

In [ ]:
v.shape